In [12]:
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

# Read csv file
data = pd.read_csv('fakedata.csv', index_col=0)

In [13]:
data

,Project Name,Address,City,LatLong,Next Meeting,Project Overview,Links,Score
1,Abra,1234 Fake Street,San Jose,"[37.348802, -121.817090]",06/02/19,Reiciendis sed officiis magni. Velit amet non ...,NaN,9.5
2,Bulbasaur,372C White Street,Santa Clara,"[37.347825, -121.942028]",06/03/19,Voluptatem consequatur et perferendis ipsa. Ve...,www.jfalds.org,2.4
3,Charmander,51 W West Avenue,San Jose,"[37.322077, -121.818726]",NaN,Alias in quis laboriosam esse velit iste. Et n...,www.isafd.com,7.4
4,Dragonair,9531 Myrtle Street,San Jose,"[37.312140, -121.828403]",NaN,Qui excepturi aut similique expedita. Libero d...,www.ufeoia.org,9.8
5,Eevee,93 Briarwood Street,Santa Clara,"[37.343011, -121.921368]",06/28/19,Qui rerum qui atque omnis soluta repudiandae n...,NaN,6.5
6,Fearow,29 Wintergreen Court,San Jose,"[37.360850, -121.925117]",07/27/19,Libero impedit aut voluptatem magni quis exerc...,NaN,3.4
7,Growlithe,392 E. Eastingdale Road,San Jose,"[37.297469, -121.869124]",NaN,Eum placeat error sed a. Non eius aspernatur d...,www.ugreaifs.com,4.6
8,Houndoom,462 Highting Court,San Jose,"[37.306700, -121.883838]",01/15/20,Sed sit id dolore sequi distinctio at explicab...,www.foisk.org,7.3
9,Ivysaur,938 Jump Street,Santa Clara,"[37.326633, -121.936609]",06/30/19,Incidunt quod eaque amet sed. Dolorem cumque v...,NaN,3.3
10,Jolteon,412 Court Drive,Santa Clara,"[37.365854, -122.017793]",08/14/19,Ut quis maiores quidem rem. Qui incidunt volup...,NaN,8.2


In [14]:
# Rename dataframe columns
data.columns = ['project_name', 'address', 'city', 'latlong', 'next_meeting', 
                'project_overview', 'links', 'score']

In [15]:
# Create engine to SQLite database file
engine = create_engine('sqlite:///projects.sqlite')

In [16]:
# Create a connection to the engine called 'conn'
conn = engine.connect()

In [17]:
# Use `declarative_base` from SQLAlchemy to model the table as an ORM class
Base = declarative_base()

class Projects(Base):
    __tablename__ = 'projects'
    
    id = Column(Integer, primary_key=True)
    project_name = Column(String)
    address = Column(String)
    city = Column(String)
    latlong = Column(String)
    next_meeting = Column(String)
    project_overview = Column(String)
    links = Column(String)
    score = Column(Integer)
    
    def __repr__(self):
        return '<Projects %r>' % (self.name)

In [18]:
# Use `create_all` to create the table in the database
Base.metadata.create_all(engine)

In [19]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
# Save the reference to the 'projects' table
project_table = sqlalchemy.Table('projects', metadata, autoload=True)

In [21]:
conn.execute(project_table.delete())

In [22]:
data = data.to_dict(orient = 'records')

In [23]:
conn.execute(project_table.insert(), data)

In [24]:
# Test that the insert works by fetching the first 5 rows
conn.execute('select * from projects limit 5').fetchall()

[(1, 'Abra', '1234 Fake Street', 'San Jose', '[37.348802, -121.817090]', '06/02/19', 'Reiciendis sed officiis magni. Velit amet non sed autem molestiae. Et aspernatur non reiciendis quia qui. Velit sed et voluptatem harum asperiores.', None, 9.5),
 (2, 'Bulbasaur', '372C White Street', 'Santa Clara', '[37.347825, -121.942028]', '06/03/19', 'Voluptatem consequatur et perferendis ipsa. Vel ut aut dicta quod est. Porro qui asperiores et cumque qui. Ab veniam officiis amet dolorum.', 'www.jfalds.org', 2.4),
 (3, 'Charmander', '51 W West Avenue', 'San Jose', '[37.322077, -121.818726]', None, 'Alias in quis laboriosam esse velit iste. Et necessitatibus quia sit voluptatem quaerat assumenda dolorum. Ipsum natus praesentium vel et quibusdam iste qui. Eum sequi unde et.', 'www.isafd.com', 7.4),
 (4, 'Dragonair', '9531 Myrtle Street', 'San Jose', '[37.312140, -121.828403]', None, 'Qui excepturi aut similique expedita. Libero distinctio et dignissimos fugit qui. Sequi maiores quas a earum quae ne

In [25]:
# Pull the data and the column information from the database
query = conn.execute('select * from projects')
db_data = query.fetchall()
db_cols = query.keys()
print(len(db_data))

10


In [27]:
# Format the rows into the geojson definition
import json

def row_to_feature(colnames, row):
    geo = {"type": "Point"}
    geo["coordinates"] = json.loads(row[4])
    props = {}
    for i in range(len(colnames)):
        if colnames[i] != "latlong":
            props[colnames[i]] = row[i]
    return {"type": "Feature", "geometry":geo, "properties":props}

data_points = []
for i in range(len(db_data)):
        data_points.append(row_to_feature(db_cols, db_data[i]))
data_dict = {"types": "FeatureCollection", "features": data_points}

In [28]:
# Write the geojson file
with open('exdata.json', 'w') as f:
    json.dump(data_dict, f, indent="\t")